# Encontrando insights de la UEFA Champions League

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

### Requerimientos:  
-Puedes utilizar SQL o PySpark

In [0]:
from pyspark.sql import SparkSession

# Iniciar una sesión de Spark
spark = SparkSession.builder.appName("UEFA_Analysis").getOrCreate()

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

Primero, cargar el archivo CSV de la temporada 2020-2021 en un DataFrame (DF):

In [0]:
# Cargar el archivo en un DF
df_2020 = spark.read.option("header", "true") \
                    .option("inferSchema", "true") \
                    .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_2/uefa_2020.csv")

# Mostrar el esquema y el contenido del DF
df_2020.printSchema()
df_2020.display()

root
 |-- STAGE: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- PENS: boolean (nullable = true)
 |-- PENS_HOME_SCORE: boolean (nullable = true)
 |-- PENS_AWAY_SCORE: boolean (nullable = true)
 |-- TEAM_NAME_HOME: string (nullable = true)
 |-- TEAM_NAME_AWAY: string (nullable = true)
 |-- TEAM_HOME_SCORE: integer (nullable = true)
 |-- TEAM_AWAY_SCORE: integer (nullable = true)
 |-- POSSESSION_HOME: string (nullable = true)
 |-- POSSESSION_AWAY: string (nullable = true)
 |-- TOTAL_SHOTS_HOME: integer (nullable = true)
 |-- TOTAL_SHOTS_AWAY: integer (nullable = true)
 |-- SHOTS_ON_TARGET_HOME: integer (nullable = true)
 |-- SHOTS_ON_TARGET_AWAY: integer (nullable = true)
 |-- DUELS_WON_HOME: string (nullable = true)
 |-- DUELS_WON_AWAY: string (nullable = true)
 |-- PREDICTION_TEAM_HOME_WIN: string (nullable = true)
 |-- PREDICTION_DRAW: string (nullable = true)
 |-- PREDICTION_TEAM_AWAY_WIN: string (nullable = true)
 |-- LOCATION: string (nullable = true)



STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION
Final,29-05-2021,false,false,false,Manchester City,Chelsea,0,1,61%,39%,7,8,1,2,50%,50%,55%,3%,42%,Estádio Do Dragão
Semi-finals second leg,05-05-2021,false,false,false,Chelsea,Real Madrid,2,0,32%,68%,15,7,5,5,52%,49%,43%,3%,54%,Stamford Bridge
Semi-finals second leg,04-05-2021,false,false,false,Manchester City,PSG,2,0,44%,56%,12,14,5,0,53%,47%,44%,3%,53%,Etihad Stadium
Semi-finals first leg,28-04-2021,false,false,false,PSG,Manchester City,1,2,40%,60%,10,11,4,6,47%,53%,65%,4%,31%,Parc des Princes
Semi-finals first leg,27-04-2021,false,false,false,Real Madrid,Chelsea,1,1,51%,49%,9,11,1,5,49%,52%,61%,4%,35%,Alfredo Di Stéfano
Quarter-finals second leg,14-04-2021,false,false,false,Liverpool,Real Madrid,0,0,57%,43%,15,6,4,2,50%,50%,34%,3%,63%,Anfield
Quarter-finals second leg,14-04-2021,false,false,false,Borussia Dortmund,Manchester City,1,2,38%,62%,10,17,3,5,42%,58%,53%,3%,44%,SIGNAL IDUNA PARK
Quarter-finals second leg,13-04-2021,false,false,false,PSG,Bayern Munich,0,1,44%,56%,10,14,3,5,54%,46%,55%,3%,42%,Parc des Princes
Quarter-finals second leg,13-04-2021,false,false,false,Chelsea,FC Porto,0,1,46%,54%,8,8,1,2,52%,48%,78%,2%,20%,Ramón Sánchez-Pizjuán
Quarter-finals first leg,07-04-2021,false,false,false,FC Porto,Chelsea,0,2,39%,61%,12,6,5,3,62%,38%,26%,3%,71%,Ramón Sánchez-Pizjuán


Utilizar las funciones `groupBy`, `agg` y `sum` para agrupar las filas según el nombre del equipo local, y añadir otra columna que sea la suma de `TEAM_HOME_SCORE` para ese nombre de equipo, ess decir, el total de goles que ha anotado el equipo jugando como local.

Después, utilizar `orderBy` con las opciones `desc` y `limit(3)` para ordenar los resultados según los 3 valores más altos de `TEAM_HOME_SCORE`.

In [0]:
from pyspark.sql.functions import sum, col

# Agrupar por TEAM_NAME_HOME y sumar los goles de cada equipo en casa
df_2020 = df_2020.groupBy("TEAM_NAME_HOME").agg(sum("TEAM_HOME_SCORE").alias("TEAM_HOME_SCORE"))

# Ordenar por el total de goles en casa en orden descendente y seleccionar los primeros 3
df_2020 = df_2020.orderBy(col("TEAM_HOME_SCORE").desc()).limit(3)

df_2020.display()

TEAM_NAME_HOME,TEAM_HOME_SCORE
Manchester City,15
Real Madrid,14
Bayern Munich,13


2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

Primero, cargar el archivo CSV de la temporada 2021-2022 en un DF:

In [0]:
# Cargar el archivo en un DF
df_2021 = spark.read.option("header", "true") \
                    .option("inferSchema", "true") \
                    .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_2/uefa_2021.csv")

# Mostrar el esquema y el contenido del DF
df_2021.printSchema()
df_2021.display()

root
 |-- STAGE: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- PENS: boolean (nullable = true)
 |-- PENS_HOME_SCORE: boolean (nullable = true)
 |-- PENS_AWAY_SCORE: boolean (nullable = true)
 |-- TEAM_NAME_HOME: string (nullable = true)
 |-- TEAM_NAME_AWAY: string (nullable = true)
 |-- TEAM_HOME_SCORE: integer (nullable = true)
 |-- TEAM_AWAY_SCORE: integer (nullable = true)
 |-- POSSESSION_HOME: string (nullable = true)
 |-- POSSESSION_AWAY: string (nullable = true)
 |-- TOTAL_SHOTS_HOME: integer (nullable = true)
 |-- TOTAL_SHOTS_AWAY: integer (nullable = true)
 |-- SHOTS_ON_TARGET_HOME: integer (nullable = true)
 |-- SHOTS_ON_TARGET_AWAY: integer (nullable = true)
 |-- DUELS_WON_HOME: string (nullable = true)
 |-- DUELS_WON_AWAY: string (nullable = true)
 |-- PREDICTION_TEAM_HOME_WIN: string (nullable = true)
 |-- PREDICTION_DRAW: integer (nullable = true)
 |-- PREDICTION_TEAM_AWAY_WIN: string (nullable = true)
 |-- LOCATION: string (nullable = true)



STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION
Final,2022-05-29,false,false,false,Liverpool,Real Madrid,0,1,50%,50%,23,3,15,2,37%,63%,47%,2,53%,Anfield
Semi-finals second leg,2022-05-05,false,false,false,Real Madrid,Man City,3,1,44%,56%,14,15,5,10,47%,53%,49%,10,51%,Santiago Bernabeu
Semi-finals second leg,2022-05-04,false,false,false,Villarreal,Liverpool,2,3,44%,56%,6,15,2,5,49%,51%,49%,4,51%,El Madrigal
Semi-finals first leg,2022-04-28,false,false,false,Liverpool,Villarreal,2,0,74%,26%,19,1,5,0,41%,59%,50%,4,50%,Anfield
Semi-finals first leg,2022-04-27,false,false,false,Man City,Real Madrid,4,3,60%,40%,16,11,6,5,46%,54%,48%,5,52%,Etihad
Quarter-finals second leg,2022-04-14,false,false,false,Atl�tico de Madrid,Man City,0,0,39%,61%,14,10,3,1,49%,51%,45%,8,55%,Civitas Metropolitan Stadium
Quarter-finals second leg,2022-04-14,false,false,false,Liverpool,Benfica,3,3,67%,33%,17,6,6,4,38%,62%,45%,9,55%,Anfield
Quarter-finals second leg,2022-04-13,false,false,false,Real Madrid,Chelsea,2,3,43%,57%,10,28,4,7,37%,63%,40%,4,60%,Santiago Bernabeu
Quarter-finals second leg,2022-04-13,false,false,false,Bayern,Villarreal,1,1,68%,32%,23,4,4,1,38%,62%,38%,7,62%,Allianz Arena
Quarter-finals first leg,2022-04-07,false,false,false,Villarreal,Bayern,1,0,38%,62%,12,22,1,4,38%,62%,47%,5,53%,El Madrigal


Se utilizan las funciones `when` y `otherwise` para crear dos nuevas columnas, `MAJORITY_POSSESSION_HOME` y `MAJORITY_POSSESSION_AWAY`, que son `1` si el equipo tuvo posesión mayoritaria y `0` si no.

In [0]:
# Comparar las posesiones y filtrar los partidos con posesión mayoritaria
df_2021 = df_2021.withColumn(
    "MAJORITY_POSSESSION_HOME", 
    when(col("POSSESSION_HOME") > col("POSSESSION_AWAY"), 1).otherwise(0)
).withColumn(
    "MAJORITY_POSSESSION_AWAY", 
    when(col("POSSESSION_AWAY") > col("POSSESSION_HOME"), 1).otherwise(0)
)

df_2021.printSchema()
df_2021.display()

root
 |-- STAGE: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- PENS: boolean (nullable = true)
 |-- PENS_HOME_SCORE: boolean (nullable = true)
 |-- PENS_AWAY_SCORE: boolean (nullable = true)
 |-- TEAM_NAME_HOME: string (nullable = true)
 |-- TEAM_NAME_AWAY: string (nullable = true)
 |-- TEAM_HOME_SCORE: integer (nullable = true)
 |-- TEAM_AWAY_SCORE: integer (nullable = true)
 |-- POSSESSION_HOME: string (nullable = true)
 |-- POSSESSION_AWAY: string (nullable = true)
 |-- TOTAL_SHOTS_HOME: integer (nullable = true)
 |-- TOTAL_SHOTS_AWAY: integer (nullable = true)
 |-- SHOTS_ON_TARGET_HOME: integer (nullable = true)
 |-- SHOTS_ON_TARGET_AWAY: integer (nullable = true)
 |-- DUELS_WON_HOME: string (nullable = true)
 |-- DUELS_WON_AWAY: string (nullable = true)
 |-- PREDICTION_TEAM_HOME_WIN: string (nullable = true)
 |-- PREDICTION_DRAW: integer (nullable = true)
 |-- PREDICTION_TEAM_AWAY_WIN: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- MAJORITY

STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION,MAJORITY_POSSESSION_HOME,MAJORITY_POSSESSION_AWAY
Final,2022-05-29,false,false,false,Liverpool,Real Madrid,0,1,50%,50%,23,3,15,2,37%,63%,47%,2,53%,Anfield,0,0
Semi-finals second leg,2022-05-05,false,false,false,Real Madrid,Man City,3,1,44%,56%,14,15,5,10,47%,53%,49%,10,51%,Santiago Bernabeu,0,1
Semi-finals second leg,2022-05-04,false,false,false,Villarreal,Liverpool,2,3,44%,56%,6,15,2,5,49%,51%,49%,4,51%,El Madrigal,0,1
Semi-finals first leg,2022-04-28,false,false,false,Liverpool,Villarreal,2,0,74%,26%,19,1,5,0,41%,59%,50%,4,50%,Anfield,1,0
Semi-finals first leg,2022-04-27,false,false,false,Man City,Real Madrid,4,3,60%,40%,16,11,6,5,46%,54%,48%,5,52%,Etihad,1,0
Quarter-finals second leg,2022-04-14,false,false,false,Atl�tico de Madrid,Man City,0,0,39%,61%,14,10,3,1,49%,51%,45%,8,55%,Civitas Metropolitan Stadium,0,1
Quarter-finals second leg,2022-04-14,false,false,false,Liverpool,Benfica,3,3,67%,33%,17,6,6,4,38%,62%,45%,9,55%,Anfield,1,0
Quarter-finals second leg,2022-04-13,false,false,false,Real Madrid,Chelsea,2,3,43%,57%,10,28,4,7,37%,63%,40%,4,60%,Santiago Bernabeu,0,1
Quarter-finals second leg,2022-04-13,false,false,false,Bayern,Villarreal,1,1,68%,32%,23,4,4,1,38%,62%,38%,7,62%,Allianz Arena,1,0
Quarter-finals first leg,2022-04-07,false,false,false,Villarreal,Bayern,1,0,38%,62%,12,22,1,4,38%,62%,47%,5,53%,El Madrigal,0,1


Usar `filter` para seleccionar solo aquellos partidos donde uno de los equipos tuvo posesión mayoritaria.

In [0]:
print(f"Registros antes de aplicar el filtro: {df_2021.count()}")

# Filtrar solo los partidos donde un equipo tuvo posesión mayoritaria
df_2021 = df_2021.filter(
    (col("MAJORITY_POSSESSION_HOME") == 1) | (col("MAJORITY_POSSESSION_AWAY") == 1)
)

print(f"Registros después de aplicar el filtro: {df_2021.count()}")

Registros antes de aplicar el filtro: 129
Registros después de aplicar el filtro: 116


Seleccionar el nombre del equipo que tuvo posesión mayoritaria en cada partido usando `when`, y luego contar la cantidad de veces que aparece cada equipo.

In [0]:
# Contar cuántas veces tuvo posesión mayoritaria cada equipo
df_2021 = df_2021.select(
    when(col("MAJORITY_POSSESSION_HOME") == 1, col("TEAM_NAME_HOME"))
    .when(col("MAJORITY_POSSESSION_AWAY") == 1, col("TEAM_NAME_AWAY"))
    .alias("TEAM_NAME")
).groupBy("TEAM_NAME").count().withColumnRenamed("count", "GAME_COUNT")

# Ordenar por la cantidad de partidos (GAME_COUNT) en orden descendente y tomar el primer equipo
df_2021 = df_2021.orderBy(col("GAME_COUNT").desc()).limit(1)

df_2021.display()

TEAM_NAME,GAME_COUNT
Liverpool,9


3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

Primero, cargar el archivo CSV de la temporada 2022-2023 en un DF:

In [0]:
# Cargar el archivo en un DF
df_2022 = spark.read.option("header", "true") \
                    .option("inferSchema", "true") \
                    .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_2/uefa_2022.csv")

# Mostrar el esquema y el contenido del DF
df_2022.printSchema()
df_2022.display()

root
 |-- STAGE: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- PENS: boolean (nullable = true)
 |-- PENS_HOME_SCORE: boolean (nullable = true)
 |-- PENS_AWAY_SCORE: boolean (nullable = true)
 |-- TEAM_NAME_HOME: string (nullable = true)
 |-- TEAM_NAME_AWAY: string (nullable = true)
 |-- TEAM_HOME_SCORE: integer (nullable = true)
 |-- TEAM_AWAY_SCORE: integer (nullable = true)
 |-- POSSESSION_HOME: string (nullable = true)
 |-- POSSESSION_AWAY: string (nullable = true)
 |-- TOTAL_SHOTS_HOME: integer (nullable = true)
 |-- TOTAL_SHOTS_AWAY: integer (nullable = true)
 |-- SHOTS_ON_TARGET_HOME: integer (nullable = true)
 |-- SHOTS_ON_TARGET_AWAY: integer (nullable = true)
 |-- DUELS_WON_HOME: string (nullable = true)
 |-- DUELS_WON_AWAY: string (nullable = true)
 |-- PREDICTION_TEAM_HOME_WIN: string (nullable = true)
 |-- PREDICTION_DRAW: integer (nullable = true)
 |-- PREDICTION_TEAM_AWAY_WIN: string (nullable = true)
 |-- LOCATION: string (nullable = true)



STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION
Group stage: Matchday 1,2022-09-06,false,false,false,Dinamo Zagreb,Chelsea,1,0,31%,69%,6,15,3,3,44%,56%,49%,10,51%,Maksimir
Group stage: Matchday 1,2022-09-06,false,false,false,Salzburg,AC Milan,1,1,34%,66%,16,14,4,3,38%,62%,41%,9,59%,Red Bull Arena
Group stage: Matchday 1,2022-09-06,false,false,false,Celtic,Real Madrid,0,3,35%,65%,10,12,4,6,46%,54%,47%,8,53%,Celtic Park
Group stage: Matchday 1,2022-09-06,false,false,false,RB Leipzig,Shakhtar Donetsk,1,4,68%,32%,13,5,2,4,47%,53%,49%,8,51%,Red Bull Arena
Group stage: Matchday 1,2022-09-06,false,false,false,Borussia Dortmund,K�benhavn,3,0,63%,37%,15,7,4,2,49%,51%,45%,6,55%,Signal Iduna Park
Group stage: Matchday 1,2022-09-06,false,false,false,Sevilla,Manchester City,0,4,38%,62%,6,24,1,10,42%,58%,43%,2,57%,Ram�n S�nchez Pizju�n
Group stage: Matchday 1,2022-09-06,false,false,false,Paris Saint-Germain,Juventus,2,1,58%,42%,15,13,6,4,36%,64%,45%,3,55%,Parc des Princes
Group stage: Matchday 1,2022-09-06,false,false,false,Benfica,Maccabi Haifa,2,0,57%,43%,14,7,5,2,46%,54%,42%,8,58%,Stadion da Luz
Group stage: Matchday 1,2022-09-07,false,false,false,Ajax,Rangers,4,0,76%,24%,17,4,9,1,47%,53%,45%,3,55%,Amsterdam Arena
Group stage: Matchday 1,2022-09-07,false,false,false,Napoli,Liverpool,4,1,38%,62%,18,15,9,7,44%,56%,45%,8,55%,San Paolo


Primero, se debe eliminar el signo de porcentaje (%) para los valores de `DUELS_WON_HOME` y `DUELS_WON_AWAY` y convertir los valores a números para poder hacer comparaciones.

In [0]:
from pyspark.sql.functions import regexp_replace

# Convertir los porcentajes en las columnas 'DUELS_WON_HOME' y 'DUELS_WON_AWAY' a valores numéricos
df_2022 = df_2022.withColumn(
    "DUELS_WON_HOME_PERCENT", 
    regexp_replace(col("DUELS_WON_HOME"), "%", "").cast("float") / 100
)
df_2022 = df_2022.withColumn(
    "DUELS_WON_AWAY_PERCENT", 
    regexp_replace(col("DUELS_WON_AWAY"), "%", "").cast("float") / 100
)

df_2022.printSchema()
df_2022.display()

root
 |-- STAGE: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- PENS: boolean (nullable = true)
 |-- PENS_HOME_SCORE: boolean (nullable = true)
 |-- PENS_AWAY_SCORE: boolean (nullable = true)
 |-- TEAM_NAME_HOME: string (nullable = true)
 |-- TEAM_NAME_AWAY: string (nullable = true)
 |-- TEAM_HOME_SCORE: integer (nullable = true)
 |-- TEAM_AWAY_SCORE: integer (nullable = true)
 |-- POSSESSION_HOME: string (nullable = true)
 |-- POSSESSION_AWAY: string (nullable = true)
 |-- TOTAL_SHOTS_HOME: integer (nullable = true)
 |-- TOTAL_SHOTS_AWAY: integer (nullable = true)
 |-- SHOTS_ON_TARGET_HOME: integer (nullable = true)
 |-- SHOTS_ON_TARGET_AWAY: integer (nullable = true)
 |-- DUELS_WON_HOME: string (nullable = true)
 |-- DUELS_WON_AWAY: string (nullable = true)
 |-- PREDICTION_TEAM_HOME_WIN: string (nullable = true)
 |-- PREDICTION_DRAW: integer (nullable = true)
 |-- PREDICTION_TEAM_AWAY_WIN: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- DUELS_WO

STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION,DUELS_WON_HOME_PERCENT,DUELS_WON_AWAY_PERCENT
Group stage: Matchday 1,2022-09-06,false,false,false,Dinamo Zagreb,Chelsea,1,0,31%,69%,6,15,3,3,44%,56%,49%,10,51%,Maksimir,0.44,0.56
Group stage: Matchday 1,2022-09-06,false,false,false,Salzburg,AC Milan,1,1,34%,66%,16,14,4,3,38%,62%,41%,9,59%,Red Bull Arena,0.38,0.62
Group stage: Matchday 1,2022-09-06,false,false,false,Celtic,Real Madrid,0,3,35%,65%,10,12,4,6,46%,54%,47%,8,53%,Celtic Park,0.46,0.54
Group stage: Matchday 1,2022-09-06,false,false,false,RB Leipzig,Shakhtar Donetsk,1,4,68%,32%,13,5,2,4,47%,53%,49%,8,51%,Red Bull Arena,0.47,0.53
Group stage: Matchday 1,2022-09-06,false,false,false,Borussia Dortmund,K�benhavn,3,0,63%,37%,15,7,4,2,49%,51%,45%,6,55%,Signal Iduna Park,0.49,0.51
Group stage: Matchday 1,2022-09-06,false,false,false,Sevilla,Manchester City,0,4,38%,62%,6,24,1,10,42%,58%,43%,2,57%,Ram�n S�nchez Pizju�n,0.42,0.58
Group stage: Matchday 1,2022-09-06,false,false,false,Paris Saint-Germain,Juventus,2,1,58%,42%,15,13,6,4,36%,64%,45%,3,55%,Parc des Princes,0.36,0.64
Group stage: Matchday 1,2022-09-06,false,false,false,Benfica,Maccabi Haifa,2,0,57%,43%,14,7,5,2,46%,54%,42%,8,58%,Stadion da Luz,0.46,0.54
Group stage: Matchday 1,2022-09-07,false,false,false,Ajax,Rangers,4,0,76%,24%,17,4,9,1,47%,53%,45%,3,55%,Amsterdam Arena,0.47,0.53
Group stage: Matchday 1,2022-09-07,false,false,false,Napoli,Liverpool,4,1,38%,62%,18,15,9,7,44%,56%,45%,8,55%,San Paolo,0.44,0.56


Comparar los valores de los porcentajes de los duelos ganados de cada equipo y decidir si un equipo ganó el duelo.

In [0]:
# Condición para identificar el equipo que ganó el duelo pero perdió el partido
df_2022 = df_2022.withColumn(
    "TEAM_LOST", 
    when(
        (col("DUELS_WON_HOME_PERCENT") > col("DUELS_WON_AWAY_PERCENT")) & \
            (col("TEAM_HOME_SCORE") < col("TEAM_AWAY_SCORE")), col("TEAM_NAME_HOME")
    ).when(
        (col("DUELS_WON_AWAY_PERCENT") > col("DUELS_WON_HOME_PERCENT")) & \
            (col("TEAM_AWAY_SCORE") < col("TEAM_HOME_SCORE")), col("TEAM_NAME_AWAY")
    ).otherwise("(DRAW)")  # Mostrar que hubo empate
)

df_2022.printSchema()
df_2022.display()

root
 |-- STAGE: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- PENS: boolean (nullable = true)
 |-- PENS_HOME_SCORE: boolean (nullable = true)
 |-- PENS_AWAY_SCORE: boolean (nullable = true)
 |-- TEAM_NAME_HOME: string (nullable = true)
 |-- TEAM_NAME_AWAY: string (nullable = true)
 |-- TEAM_HOME_SCORE: integer (nullable = true)
 |-- TEAM_AWAY_SCORE: integer (nullable = true)
 |-- POSSESSION_HOME: string (nullable = true)
 |-- POSSESSION_AWAY: string (nullable = true)
 |-- TOTAL_SHOTS_HOME: integer (nullable = true)
 |-- TOTAL_SHOTS_AWAY: integer (nullable = true)
 |-- SHOTS_ON_TARGET_HOME: integer (nullable = true)
 |-- SHOTS_ON_TARGET_AWAY: integer (nullable = true)
 |-- DUELS_WON_HOME: string (nullable = true)
 |-- DUELS_WON_AWAY: string (nullable = true)
 |-- PREDICTION_TEAM_HOME_WIN: string (nullable = true)
 |-- PREDICTION_DRAW: integer (nullable = true)
 |-- PREDICTION_TEAM_AWAY_WIN: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- DUELS_WO

STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION,DUELS_WON_HOME_PERCENT,DUELS_WON_AWAY_PERCENT,TEAM_LOST
Group stage: Matchday 1,2022-09-06,false,false,false,Dinamo Zagreb,Chelsea,1,0,31%,69%,6,15,3,3,44%,56%,49%,10,51%,Maksimir,0.44,0.56,Chelsea
Group stage: Matchday 1,2022-09-06,false,false,false,Salzburg,AC Milan,1,1,34%,66%,16,14,4,3,38%,62%,41%,9,59%,Red Bull Arena,0.38,0.62,(DRAW)
Group stage: Matchday 1,2022-09-06,false,false,false,Celtic,Real Madrid,0,3,35%,65%,10,12,4,6,46%,54%,47%,8,53%,Celtic Park,0.46,0.54,(DRAW)
Group stage: Matchday 1,2022-09-06,false,false,false,RB Leipzig,Shakhtar Donetsk,1,4,68%,32%,13,5,2,4,47%,53%,49%,8,51%,Red Bull Arena,0.47,0.53,(DRAW)
Group stage: Matchday 1,2022-09-06,false,false,false,Borussia Dortmund,K�benhavn,3,0,63%,37%,15,7,4,2,49%,51%,45%,6,55%,Signal Iduna Park,0.49,0.51,K�benhavn
Group stage: Matchday 1,2022-09-06,false,false,false,Sevilla,Manchester City,0,4,38%,62%,6,24,1,10,42%,58%,43%,2,57%,Ram�n S�nchez Pizju�n,0.42,0.58,(DRAW)
Group stage: Matchday 1,2022-09-06,false,false,false,Paris Saint-Germain,Juventus,2,1,58%,42%,15,13,6,4,36%,64%,45%,3,55%,Parc des Princes,0.36,0.64,Juventus
Group stage: Matchday 1,2022-09-06,false,false,false,Benfica,Maccabi Haifa,2,0,57%,43%,14,7,5,2,46%,54%,42%,8,58%,Stadion da Luz,0.46,0.54,Maccabi Haifa
Group stage: Matchday 1,2022-09-07,false,false,false,Ajax,Rangers,4,0,76%,24%,17,4,9,1,47%,53%,45%,3,55%,Amsterdam Arena,0.47,0.53,Rangers
Group stage: Matchday 1,2022-09-07,false,false,false,Napoli,Liverpool,4,1,38%,62%,18,15,9,7,44%,56%,45%,8,55%,San Paolo,0.44,0.56,Liverpool


Usamos `filter` para mantener solo las filas donde se cumple la condición de que un equipo ganó el duelo pero perdió el partido, y seleccionar las columnas `STAGE` y `TEAM_LOST`

In [0]:
# Filtrar solo los registros donde hay un equipo que ganó el duelo pero perdió el partido
df_2022 = df_2022.filter(col("TEAM_LOST") != "(DRAW)")

# Seleccionar las columnas relevantes y mostrar los resultados por fase (STAGE)
df_2022 = df_2022.select("STAGE", "TEAM_LOST")

df_2022.display()

STAGE,TEAM_LOST
Group stage: Matchday 1,Chelsea
Group stage: Matchday 1,K�benhavn
Group stage: Matchday 1,Juventus
Group stage: Matchday 1,Maccabi Haifa
Group stage: Matchday 1,Rangers
Group stage: Matchday 1,Liverpool
Group stage: Matchday 1,Porto
Group stage: Matchday 1,Bayer Leverkusen
Group stage: Matchday 1,Viktoria Plze?
Group stage: Matchday 1,Marseille
